In [2]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 19: Beacon Scanner ---</h2><p>As your <a href="17">probe</a> drifted down through this area, it released an assortment of <em>beacons</em> and <em>scanners</em> into the water. It's difficult to navigate in the pitch black open waters of the ocean trench, but if you can build a map of the trench using data from the scanners, you should be able to safely reach the bottom.</p>
<p>The beacons and scanners float motionless in the water; they're designed to maintain the same position for long periods of time. Each scanner is capable of detecting all beacons in a large cube centered on the scanner; beacons that are at most 1000 units away from the scanner in each of the three axes (<code>x</code>, <code>y</code>, and <code>z</code>) have their precise position determined relative to the scanner. However, scanners cannot detect other scanners. The submarine has automatically summarized the relative positions of beacons detected by each scanner (your puzzle input).</p>
<p>For example, if a scanner is at <code>x,y,z</code> coordinates <code>500,0,-500</code> and there are beacons at <code>-500,1000,-1500</code> and <code>1501,0,-500</code>, the scanner could report that the first beacon is at <code>-1000,1000,-1000</code> (relative to the scanner) but would not detect the second beacon at all.</p>
<p>Unfortunately, while each scanner can report the positions of all detected beacons relative to itself, <em>the scanners do not know their own position</em>. You'll need to determine the positions of the beacons and scanners yourself.</p>
<p>The scanners and beacons map a single contiguous 3d region. This region can be reconstructed by finding pairs of scanners that have overlapping detection regions such that there are <em>at least 12 beacons</em> that both scanners detect within the overlap. By establishing 12 common beacons, you can precisely determine where the scanners are relative to each other, allowing you to reconstruct the beacon map one scanner at a time.</p>
<p>For a moment, consider only two dimensions. Suppose you have the following scanner reports:</p>
<pre><code>--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1
</code></pre>

<p>Drawing <code>x</code> increasing rightward, <code>y</code> increasing upward, scanners as <code>S</code>, and beacons as <code>B</code>, scanner <code>0</code> detects this:</p>
<pre><code>...B.
B....
....B
S....
</code></pre>
<p>Scanner <code>1</code> detects this:</p>
<pre><code>...B..
B....S
....B.
</code></pre>
<p>For this example, assume scanners only need 3 overlapping beacons. Then, the beacons visible to both scanners overlap to produce the following complete map:</p>
<pre><code>...B..
B....S
....B.
S.....
</code></pre>
<p>Unfortunately, there's a second problem: the scanners also don't know their <em>rotation or facing direction</em>. Due to magnetic alignment, each scanner is rotated some integer number of 90-degree turns around all of the <code>x</code>, <code>y</code>, and <code>z</code> axes. That is, one scanner might call a direction positive <code>x</code>, while another scanner might call that direction negative <code>y</code>. Or, two scanners might agree on which direction is positive <code>x</code>, but one scanner might be upside-down from the perspective of the other scanner. In total, each scanner could be in any of 24 different orientations: facing positive or negative <code>x</code>, <code>y</code>, or <code>z</code>, and considering any of four directions "up" from that facing.</p>
<p>For example, here is an arrangement of beacons as seen from a scanner in the same position but in different orientations:</p>
<pre><code>--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,z
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8
</code></pre>

<p>By finding pairs of scanners that both see at least 12 of the same beacons, you can assemble the entire map. For example, consider the following report:</p>
<pre><code>--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
</code></pre>

<p>Because all coordinates are relative, in this example, all "absolute" positions will be expressed relative to scanner <code>0</code> (using the orientation of scanner <code>0</code> and as if scanner <code>0</code> is at coordinates <code>0,0,0</code>).</p>
<p>Scanners <code>0</code> and <code>1</code> have overlapping detection cubes; the 12 beacons they both detect (relative to scanner <code>0</code>) are at the following coordinates:</p>
<pre><code>-618,-824,-621
-537,-823,-458
-447,-329,318
404,-588,-901
544,-627,-890
528,-643,409
-661,-816,-575
390,-675,-793
423,-701,434
-345,-311,381
459,-707,401
-485,-357,347
</code></pre>
<p>These same 12 beacons (in the same order) but from the perspective of scanner <code>1</code> are:</p>
<pre><code>686,422,578
605,423,415
515,917,-361
-336,658,858
-476,619,847
-460,603,-452
729,430,532
-322,571,750
-355,545,-477
413,935,-424
-391,539,-444
553,889,-390
</code></pre>
<p>Because of this, scanner <code>1</code> must be at <code>68,-1246,-43</code> (relative to scanner <code>0</code>).</p>
<p>Scanner <code>4</code> overlaps with scanner <code>1</code>; the 12 beacons they both detect (relative to scanner <code>0</code>) are:</p>
<pre><code>459,-707,401
-739,-1745,668
-485,-357,347
432,-2009,850
528,-643,409
423,-701,434
-345,-311,381
408,-1815,803
534,-1912,768
-687,-1600,576
-447,-329,318
-635,-1737,486
</code></pre>
<p>So, scanner <code>4</code> is at <code>-20,-1133,1061</code> (relative to scanner <code>0</code>).</p>
<p>Following this process, scanner <code>2</code> must be at <code>1105,-1205,1229</code> (relative to scanner <code>0</code>) and scanner <code>3</code> must be at <code>-92,-2380,-20</code> (relative to scanner <code>0</code>).</p>
<p>The full list of beacons (relative to scanner <code>0</code>) is:</p>
<pre><code>-892,524,684
-876,649,763
-838,591,734
-789,900,-551
-739,-1745,668
-706,-3180,-659
-697,-3072,-689
-689,845,-530
-687,-1600,576
-661,-816,-575
-654,-3158,-753
-635,-1737,486
-631,-672,1502
-624,-1620,1868
-620,-3212,371
-618,-824,-621
-612,-1695,1788
-601,-1648,-643
-584,868,-557
-537,-823,-458
-532,-1715,1894
-518,-1681,-600
-499,-1607,-770
-485,-357,347
-470,-3283,303
-456,-621,1527
-447,-329,318
-430,-3130,366
-413,-627,1469
-345,-311,381
-36,-1284,1171
-27,-1108,-65
7,-33,-71
12,-2351,-103
26,-1119,1091
346,-2985,342
366,-3059,397
377,-2827,367
390,-675,-793
396,-1931,-563
404,-588,-901
408,-1815,803
423,-701,434
432,-2009,850
443,580,662
455,729,728
456,-540,1869
459,-707,401
465,-695,1988
474,580,667
496,-1584,1900
497,-1838,-617
527,-524,1933
528,-643,409
534,-1912,768
544,-627,-890
553,345,-567
564,392,-477
568,-2007,-577
605,-1665,1952
612,-1593,1893
630,319,-379
686,-3108,-505
776,-3184,-501
846,-3110,-434
1135,-1161,1235
1243,-1093,1063
1660,-552,429
1693,-557,386
1735,-437,1738
1749,-1800,1813
1772,-405,1572
1776,-675,371
1779,-442,1789
1780,-1548,337
1786,-1538,337
1847,-1591,415
1889,-1729,1762
1994,-1805,1792
</code></pre>
<p>In total, there are <code><em>79</em></code> beacons.</p>
<p>Assemble the full map of beacons. <em>How many beacons are there?</em></p>
</article>


Solution from

> https://github.com/marcodelmastro/AdventOfCode2021/blob/main/Day19.ipynb


In [ ]:
from itertools import combinations
import scipy

type Point = tuple[int, ...]

rotation_test = """
--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8
"""


def parse(s: str) -> list[list[Point]]:
    return [
        [tuple(int(d) for d in b.split(",")) for b in sc.splitlines()[1:]]
        for sc in re.split(r"\n\s*\n", s.strip())
    ]


rotations = [
    r.astype(int)
    for r in scipy.spatial.transform.Rotation.create_group("O").as_matrix()
]

rotation_test = parse(rotation_test)

for ((i1, scan1), (i2, scan2)), R in product(
    combinations(enumerate(rotation_test), 2), rotations
):
    found_rotation = 0
    for v0, v1 in zip(scan1, scan2):
        X0 = np.array(v0)
        X1 = np.array(v1)
        X = np.matmul(R, X0)
        if (X == X1).all():
            found_rotation += 1
    if found_rotation == len(rotation_test[0]):
        print(f"Scanners{i1, i2} rotation:")
        print(R)
        print()

Solution from

> https://github.com/marcodelmastro/AdventOfCode2021/blob/main/Day19.ipynb


In [5]:
example = """
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
"""

In [ ]:
from math import dist


def find_common_beacons(beacons1, beacons2, verbose=True):
    c1 = [(a, b) for a, b in combinations(beacons1, 2)]
    c2 = [(a, b) for a, b in combinations(beacons2, 2)]
    d1 = [dist(a, b) for a, b in combinations(beacons1, 2)]
    d2 = [dist(a, b) for a, b in combinations(beacons2, 2)]

    common = list(set(d1) & set(d2))

    if len(common) >= 66:
        if verbose:
            print("Found overlapping region ", end="")
        C1 = []
        C2 = []
        for c in common:
            i1 = d1.index(c)
            i2 = d2.index(c)
            for x1 in c1[i1]:
                if x1 not in C1:
                    C1.append(x1)
            for x2 in c2[i2]:
                if x2 not in C2:
                    C2.append(x2)
        if verbose:
            print("with {} common beacons.".format(len(C1)))
        if verbose:
            print("Looking for rotation and scanner distance... ", end="")
        # Taking one point as reference
        X1 = np.array(C1[0])
        R = None
        dX = None
        # Trying all points in corresponding set to find match
        matchFound = False
        for x2 in C2:
            X2 = np.array(x2)
            # Trying all rotations
            for R in rotations:
                X2rot = np.matmul(R, X2)
                # Translation between reference point X1 and test point X2 after rotation
                dX = X1 - X2rot
                # If point and rotation correct, than all other points should have a correspondence in original set
                C2transf = [tuple(np.matmul(R, np.array(x2t)) + dX) for x2t in C2]
                inOriginal = [1 for x2t in C2transf if x2t in C1]
                # There might be more than 12 elements in C1 for spurious matching of
                # distances, only check for lenght of inOriginal to be at least 12
                # if len(inOriginal)==len(C1):
                if len(inOriginal) >= 12:
                    if verbose:
                        print("found!")
                    if verbose:
                        print(R, dX)
                    matchFound = True
                    break
            if matchFound:
                break
        if not matchFound:
            return [], (0, 0, 0)
        if verbose:
            print(
                "Translating all points in second set to coordinate of first set... ",
                end="",
            )
        beacon2new = [tuple(np.matmul(R, np.array(x2)) + dX) for x2 in beacons2]
        if verbose:
            print("Done.")
        return beacon2new, tuple(dX)
    else:
        if verbose:
            print("No overlapping region found.")
        return [], (0, 0, 0)


b1, b2, *_ = parse(example)

b2_new, Xscanner2 = find_common_beacons(b1, b2, False)

print(len(b1), len(b2), len(set(b1) | set(b2_new)))
print(Xscanner2)

In [7]:
def overlaps(scanner) -> tuple[list[list], list[tuple[int, int, int]]]:
    scanner_new = [[]] * len(scanner)
    scanner_new[0] = list(scanner[0])
    scanner_coord = [(0, 0, 0)] * len(scanner)
    translated = [0]
    refs = [0]

    while len(translated) < len(scanner):
        iref = refs.pop()
        b1 = scanner_new[iref]
        for i in range(len(scanner)):
            if i != iref and i not in translated:
                b2 = scanner[i]
                b2new, Xscanner = find_common_beacons(b1, b2, verbose=False)
                if len(b2new):
                    scanner_new[i] = b2new
                    scanner_coord[i] = Xscanner
                    translated.append(i)
                    refs.append(i)

    return scanner_new, scanner_coord

In [ ]:
def count_beacons(s: str) -> int:
    scanner_new, _ = overlaps(parse(s))
    overlap = set([])
    for b in scanner_new:
        overlap |= set(b)
    return len(overlap)


print("Test 1:", count_beacons(example))

In [ ]:
with open("../input/day19.txt") as f:
    puzzle = f.read()

print(f"Part I: {count_beacons(puzzle)}")

<link href="style.css" rel="stylesheet"></link>
<p>Your puzzle answer was <code>392</code>.</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Sometimes, it's a good idea to appreciate just how <span title="The deepest parts of the ocean are about as deep as the altitude of a normal commercial aircraft, roughly 11 kilometers or 36000 feet.">big</span> the ocean is. Using the <a href="https://en.wikipedia.org/wiki/Taxicab_geometry" target="_blank">Manhattan distance</a>, how far apart do the scanners get?</p>
<p>In the above example, scanners <code>2</code> (<code>1105,-1205,1229</code>) and <code>3</code> (<code>-92,-2380,-20</code>) are the largest Manhattan distance apart. In total, they are <code>1197 + 1175 + 1249 = <em>3621</em></code> units apart.</p>
<p><em>What is the largest Manhattan distance between any two scanners?</em></p>
</article>


In [10]:
def manhattanDistance(a: tuple[int, ...], b: tuple[int, ...]) -> int:
    return sum(abs(xa - xb) for xa, xb in zip(a, b))


def largest_manhattan_distance(s: str) -> int:
    _, coordinates = overlaps(parse(s))
    return int(max(manhattanDistance(a, b) for a, b in combinations(coordinates, 2)))


assert largest_manhattan_distance(example) == 3621

In [ ]:
print(f"Part II: { largest_manhattan_distance(puzzle) }")

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>13332</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
